**Import Libraries**

In [0]:
import pandas as pd
import numpy as np
from functools import reduce
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when
import pyspark.sql.functions as F
from pyspark.sql.functions import format_number 


**Load location data**<br>
-Drop all null values 

In [0]:
Area = spark.read.table("default.area_data").dropna()
Area.limit(5).display()

tSaddress,tDaddress,tStartLat,tStartLong,tEndLat,tEndLong
"86 Union Ave, Harare, Zimbabwe","Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.826984772255297,31.051359772682186,-17.7926959,31.0426273
"Takaruza St, Harare, Zimbabwe","1906 Borrowdale Rd, Harare, Zimbabwe",-17.915548323393555,30.96475627273321,-17.7834421,31.0796598
"1906 Borrowdale Rd, Harare, Zimbabwe","3XG4+5R Harare, Zimbabwe",-17.783321948224483,31.079784147441387,-17.9246189,30.9570884
"3XG4+2Q Harare, Zimbabwe","638H+FRX, Harare, Zimbabwe",-17.924987203496016,30.95699865370989,-17.7838185,31.0797282
"3XG4+2R Harare, Zimbabwe","1906 Borrowdale Rd, Harare, Zimbabwe",-17.924960407333128,30.957011058926582,-17.7836974,31.0797237


**Start site**<br>
-Select starting address, latitude and longtude

In [0]:

StartSite = Area.select("tSaddress", "tStartLat",
"tStartLong").limit(5)
StartSite.display()

tSaddress,tStartLat,tStartLong
"86 Union Ave, Harare, Zimbabwe",-17.826984772255297,31.051359772682186
"Takaruza St, Harare, Zimbabwe",-17.915548323393555,30.96475627273321
"1906 Borrowdale Rd, Harare, Zimbabwe",-17.783321948224483,31.079784147441387
"3XG4+2Q Harare, Zimbabwe",-17.924987203496016,30.95699865370989
"3XG4+2R Harare, Zimbabwe",-17.924960407333128,30.957011058926582


**Start site**<br>
-Select ending address, latitude and longtude

In [0]:

EndSite = Area.select("tDaddress", "tEndLat",
"tEndLong").limit(5)
EndSite.display()


tDaddress,tEndLat,tEndLong
"Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.7926959,31.0426273
"1906 Borrowdale Rd, Harare, Zimbabwe",-17.7834421,31.0796598
"3XG4+5R Harare, Zimbabwe",-17.9246189,30.9570884
"638H+FRX, Harare, Zimbabwe",-17.7838185,31.0797282
"1906 Borrowdale Rd, Harare, Zimbabwe",-17.7836974,31.0797237


**Convert starting and Ending site to SQL tables**

In [0]:
StartSite.createOrReplaceTempView("StartSite")
EndSite.createOrReplaceTempView("EndSite")

**Perfom a cross join**<br>
-Cross join starting and ending site<br>
-Table name is sites_combined

In [0]:
%sql
Create or replace temp view sites_combined as 
  select * from StartSite  cross join EndSite


In [0]:
%sql
SELECT* FROM sites_combined limit 5


tSaddress,tStartLat,tStartLong,tDaddress,tEndLat,tEndLong
"86 Union Ave, Harare, Zimbabwe",-17.826984772255297,31.051359772682186,"Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.7926959,31.0426273
"Takaruza St, Harare, Zimbabwe",-17.915548323393555,30.96475627273321,"Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.7926959,31.0426273
"1906 Borrowdale Rd, Harare, Zimbabwe",-17.783321948224483,31.079784147441387,"Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.7926959,31.0426273
"3XG4+2Q Harare, Zimbabwe",-17.924987203496016,30.95699865370989,"Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.7926959,31.0426273
"3XG4+2R Harare, Zimbabwe",-17.924960407333128,30.957011058926582,"Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.7926959,31.0426273


**Calculate x and y values**<br>
-There is need to calculate the x n y values using the code below.

In [0]:
%sql
Create or replace temp view sites_combined_one as 
  select tSaddress, tDaddress, tStartLat,tStartLong, tEndLat, tEndLong, 69.1*(tEndLat -tStartLat) x, 69.1*(tEndLong -tStartLong)*cos(tStartLat/57.3) y
  from sites_combined


**Display the table with x n y**

In [0]:
%sql
SELECT* FROM sites_combined_one limit 5

tSaddress,tDaddress,tStartLat,tStartLong,tEndLat,tEndLong,x,y
"86 Union Ave, Harare, Zimbabwe","Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.826984772255297,31.051359772682186,-17.7926959,31.0426273,2.3693610728411745,-0.5744453679232016
"86 Union Ave, Harare, Zimbabwe","1906 Borrowdale Rd, Harare, Zimbabwe",-17.826984772255297,31.051359772682186,-17.7834421,31.0796598,3.008798652841157,1.8616513554037046
"86 Union Ave, Harare, Zimbabwe","3XG4+5R Harare, Zimbabwe",-17.826984772255297,31.051359772682186,-17.9246189,30.9570884,-6.746518227158856,-6.201422590821307
"86 Union Ave, Harare, Zimbabwe","638H+FRX, Harare, Zimbabwe",-17.826984772255297,31.051359772682186,-17.7838185,31.0797282,2.9827894128411305,1.8661508900253334
"86 Union Ave, Harare, Zimbabwe","1906 Borrowdale Rd, Harare, Zimbabwe",-17.826984772255297,31.051359772682186,-17.7836974,31.0797237,2.9911574228409683,1.865854868010565


**Distance**<br>
-Calculate distance using fomula below and create the distance column

In [0]:
%sql
Create or replace temp view sites_combined_two as 
  select tSaddress,tDaddress, tStartLat, tStartLong, tEndLat,tEndLong, SQRT(POWER(x,2) + POWER(y,2))*1.60933 distance_km
  from sites_combined_one

**Display**<br>
-Visualise the calculated distance column

In [0]:
%sql
select* from sites_combined_two limit 5


tSaddress,tDaddress,tStartLat,tStartLong,tEndLat,tEndLong,distance_km
"86 Union Ave, Harare, Zimbabwe","Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.826984772255297,31.051359772682186,-17.7926959,31.0426273,3.923551614278735
"Takaruza St, Harare, Zimbabwe","Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.915548323393555,30.96475627273321,-17.7926959,31.0426273,15.95424728520021
"1906 Borrowdale Rd, Harare, Zimbabwe","Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.783321948224483,31.079784147441387,-17.7926959,31.0426273,4.070358011974687
"3XG4+2Q Harare, Zimbabwe","Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.924987203496016,30.95699865370989,-17.7926959,31.0426273,17.277509453648197
"3XG4+2R Harare, Zimbabwe","Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.924960407333128,30.957011058926582,-17.7926959,31.0426273,17.27428459616109


**Convert site_combined_two df above from SQL to pyspark df**

In [0]:

df = sqlContext.sql("select * from sites_combined_two")
  

**Final df**<br>
-The final df is in pyspark and the distance calculated has been rounded to one decimal form

In [0]:
df = df.withColumn('Distance KM', (format_number('distance_km', 1))).drop('distance_km')

df.limit(5).display()                  
                  

tSaddress,tDaddress,tStartLat,tStartLong,tEndLat,tEndLong,Distance KM
"86 Union Ave, Harare, Zimbabwe","Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.826984772255297,31.051359772682186,-17.7926959,31.0426273,3.9
"Takaruza St, Harare, Zimbabwe","Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.915548323393555,30.96475627273321,-17.7926959,31.0426273,16.0
"1906 Borrowdale Rd, Harare, Zimbabwe","Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.783321948224483,31.079784147441387,-17.7926959,31.0426273,4.1
"3XG4+2Q Harare, Zimbabwe","Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.924987203496016,30.95699865370989,-17.7926959,31.0426273,17.3
"3XG4+2R Harare, Zimbabwe","Kapoto Restaurant, Chelmsford Road, Harare, Zimbabwe",-17.924960407333128,30.957011058926582,-17.7926959,31.0426273,17.3
